# SAD - Synthetic Augmented Data Generator

This jupyter notebook allows a quick try out of our code.

## Table of Content

1) Preparation for Notebook

2) Creating Synthetic Data

3) Automatically augmenting real world data

4) Manually augmenting real world data


## Preparation for Notebook
Steps that are necessary before executing this notebook:
1) Create a virtual Python environment

For Linux
```bash
python3 -m venv venv4datagenerator
source venv4datagenerator/bin/activate
```
For Windows
```bash
python -m venv venv4datagenerator
venv4datagenerator\Scripts\activate.bat
```

2) Install dependencies
```
pip install -r requirements.txt
```

3) Launch Jupyter Notebook
```bash
jupyter notebook
```

## Chunks

The data generator picks the basic building blocks (chunks) from the folder `chunks` to dynamically create and annotate a ground plane. There are four different chunk types: `curve_left`, `curve_right`, `line` and `intersection`. For every chunk type there has to exist exactly one label and at least one image. If multiple images exist for one type they would be called variants of the chunk. The label contains the semantic information of the chunk. The image is the visual appearance of the road element. Every chunk type has a correspondent JSON file with its metadata. One important meta information is the ideal path that the car would take through the chunk.

This figure shows how the ground plane is modularily constructed from chunks.
<img width="50%" src="https://user-images.githubusercontent.com/88937076/138558644-222a3bcd-ea1d-46ec-918f-5033dd79b3ef.png"></img>

In our example the chunks represent the building blocks of a miniature track for a model car. By changing the chunk images, one can adapt the data generator to their own self-driving task. The chunk images need a consistent scale and size which has to be defined in the [config](CONFIG.md). Additionally the images have to be in the bird's-eye-view.

Run the following code block to see all available chunks in our demonstrator.


In [ ]:
"""Plot all available chunks."""

import cv2
import glob
import matplotlib.pyplot as plt

labels_list = glob.glob("./chunks/*label*.png")
len_list = len(labels_list)

fig = plt.figure(figsize=(5, 20))

columns = 2
rows = len_list
for i in range(1, len_list+1):
    label_path = labels_list[i-1]
    img = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    fig.add_subplot(rows, columns, 2*i)
    plt.axis('off')
    plt.imshow(img, cmap='gray')

    img_path = label_path.replace("label", "nice")
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    fig.add_subplot(rows, columns, 2*i-1)
    plt.axis('off')
    
    try:
        plt.imshow(img, cmap='gray')
    except:
        print("Following image was not found: " + str(img_path))
plt.show()


## Overlays

Overlays are random distracting images which the data generator puts on the camera image to enhance the variety of the dataset. They serve as disturbing artifacts and do not affect the labels of the images because they do not belong to a label class. The user can add their own overlays inside the folder `overlays`.

Run the following code block to see some examples of obstacles in our demonstrator.

In [ ]:
"""Plot some examples of overlays."""

import cv2
import glob
import matplotlib.pyplot as plt

overlay_list = glob.glob("./overlays/*.png")

columns = 1
rows = min(len(overlay_list), 5)

print("Example of Overlays")
fig_overlay = plt.figure(figsize=(3, 15))
for i in range(1, rows+1):
    overlay_path = overlay_list[i-1]
    img = cv2.imread(overlay_path, cv2.IMREAD_GRAYSCALE)
    fig_overlay.add_subplot(rows, columns, i)
    plt.axis('off')
    plt.imshow(img, cmap='gray')
plt.show()

## Synthetic Data Generation

Run the following code to synthesize annotated samples.

In [ ]:
import os
import time
import random
import argparse
import numpy as np

from config import Config
import road
import render
from disturbances import *
import augment
from main import *
from startline import Startline
from manual_augment import ManualAugment
config = Config("config1.json", debug=False)

if config["seed"]:
    random.seed(config["seed"])
    np.random.seed(config["seed"])

output_path_labels = config["paths"]["labels_output_path"]
output_path_images = config["paths"]["images_output_path"]

for name, split in config["splits"].items():
    os.makedirs(output_path_labels.format(splitname=name),
                exist_ok=True)
    os.makedirs(output_path_images.format(splitname=name), exist_ok=True)
    print("generating split", name)
    print("synthetic")
    print()
    idcs = list(range(split["size"]))
    if config["shuffle"]:
        random.shuffle(idcs)
    generate_synthetic(
        config,
        name,
        idcs[round(
            (1 - split["fraction_synthetic"]) *
            split["size"]):])

    print("augmented")
    generate_augmented(
        config,
        name,
        idcs[:round(
            split["fraction_augmented"] *
            split["size"])])

## Manual Data Augmentation

This repository offers an interactive GUI for manual augmentation.

The class `ManualAugment` in the file `manual_augment.py` contains generic functions for manually adding overlays to annotated samples.
The program `startline.py` shows an example of the manual augmentation. Here a start line will be added to the image and label of the data sample.
A interactive GUI is offered to the user where he can translate and rotate the overlay inside the image.
When the overlay has reached the final pose, the user can save the image. The overlay will be automatically added to the image and label.

The start line can be moved using the following controls:


| Key | Action |
|-----|--------|
| W   | move start line away from the camera  | 
| A   | move start line to the left | 
| S   | move start line towards the camera| 
| D   | move start line to the right | 
| E   | rotate the start line clockwise | 
| R   | rotate the start line counter-clockwise | 
| Q   | skip the image | 
| Space   | save the image and go to the next | 
| X   | quit | 

Keep in mind that the input and output paths for the images have to match.
The paths are set in the configuration file `config1.json`.

In [ ]:
import os
import time
import random
import argparse
import numpy as np

from config import Config
import road
import render
from disturbances import *
import augment
from main import *
from startline import Startline
from manual_augment import ManualAugment
config = Config("config1.json", debug=False)



startline = Startline()
index = 0

startline_img, startline_label = startline.draw_startline()

labels_list = glob.glob(startline.label_input_path + "/*.png") if os.path.isdir(startline.label_input_path) else glob.glob(startline.label_input_path)
images_list = glob.glob(startline.img_input_path + "/*.png") if os.path.isdir(startline.img_input_path) else glob.glob(startline.img_input_path)

startline_img_, startline_label_ = startline.create_overlay(startline_img)

if len(labels_list) == 0:
    print(f"no annotated images found under {self.label_input_path}")

if len(images_list) == 0:
    print(f"no images found under {self.img_input_path}")

exit = False

for label_path, img_path in zip(labels_list, images_list):
    if exit:
        break
    camera_img, camera_label = startline.import_annotated_data(
            img_path, label_path)
    bird_img, bird_label = startline.get_birds_eye_view(
            camera_img, camera_label)

    startline_img, startline_label = (np.copy(startline_img_),
                                     np.copy(startline_label_))
    key = 0
    while key != ord(" ") or ord("q"):

        startline_img, startline_label = startline.transform_image(
                startline_img, startline_label, key)

        (bird_img_n, bird_label_n,
         camera_img_n, camera_label_n) = startline.merge_bird_overlay(
                np.copy(startline_img), np.copy(startline_label),
                np.copy(bird_img), np.copy(bird_label),
                np.copy(camera_img), np.copy(camera_label))
        key = startline.visualize_augmentation(bird_label_n, bird_img_n,
                                               camera_img_n, camera_label_n,
                                               index)

        if key == ord(" "):
            break
        if key == ord("q"):
            break
        if key == ord("x"):
            exit = True
            cv2.destroyAllWindows()
            break
    index += 1

The JSON configuration file `config1.json` is used for most of the configuration.
The full reference is available in [CONFIG.md](CONFIG.md).

